In [ ]:
# Task-1:-  Image Quality Enhancement, Image Restoration
# Step 1: Load & Preprocess Data
# •	Load dataset (e.g., MNIST, CIFAR-10).
# •	Normalize pixel values between [0,1].
# •	Resize images if necessary.
# •	Convert to grayscale if required.

# Step 2: Simulate Degradation
# •	Introduce noise (Gaussian noise, salt & pepper noise).
# •	Apply blur (motion blur, Gaussian blur).
# •	Add compression artifacts if needed.

# Step 3: Build the Autoencoder Model
# •	Encoder: Uses convolutional layers to extract features and compress image representation.
# •	Decoder: Reconstructs the image from the compressed representation using up-sampling and convolutional layers.

# Step 4: Train the Model
# •	Train autoencoder using degraded images as input and original images as target output.
# •	Use Mean Squared Error (MSE) as the loss function.
# •	Optimize with Adam optimizer.
# •	Validate on test data.

# Step 5: Image Restoration & Evaluation
# •	Predict restored images using the trained model.
# •	Compare original, degraded, and restored images.
# •	Evaluate performance using PSNR (Peak Signal-to-Noise Ratio)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow_datasets as tfds

In [ ]:
# Load and preprocess CIFAR-100 dataset
def load_cifar100():
    (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar100.load_data()
    train_images, test_images = train_images / 255.0, test_images / 255.0
    return train_images, train_labels, test_images, test_labels

In [ ]:
# Simulate degradation (Gaussian noise, blur)
def degrade_images(images):
    degraded_images = []
    for img in images:
        noisy_img = img + 0.1 * np.random.randn(*img.shape)
        noisy_img = np.clip(noisy_img, 0, 1)
        blurred_img = cv2.GaussianBlur(noisy_img, (5, 5), 1)
        degraded_images.append(blurred_img)
    return np.array(degraded_images)

In [ ]:
# Build Autoencoder model
def build_autoencoder():
    input_img = layers.Input(shape=(32, 32, 3))
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)

    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    output_img = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = models.Model(input_img, output_img)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

In [ ]:
# Load and preprocess dataset
train_images, train_labels, test_images, test_labels = load_cifar100()
degraded_train = degrade_images(train_images)
degraded_test = degrade_images(test_images)

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [ ]:
# Train autoencoder
autoencoder = build_autoencoder()
autoencoder.fit(degraded_train, train_images, epochs=10, batch_size=64, validation_data=(degraded_test, test_images))

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 208s 263ms/step - loss: 0.0172 - val_loss: 0.0057
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 219s 280ms/step - loss: 0.0055 - val_loss: 0.0049
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 205s 262ms/step - loss: 0.0048 - val_loss: 0.0046
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 269s 271ms/step - loss: 0.0045 - val_loss: 0.0043
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 201s 258ms/step - loss: 0.0042 - val_loss: 0.0042
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 204s 261ms/step - loss: 0.0041 - val_loss: 0.0040
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 212s 271ms/step - loss: 0.0039 - val_loss: 0.0039
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 262s 271ms/step - loss: 0.0039 - val_loss: 0.0038
Epoch 9/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 252s 258ms/step - loss: 0.0038 - val_loss: 0.0038
Epoch 10/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 264s 260ms/step - loss: 0.0037 - val_loss: 0.0037


In [ ]:
# Restore images
restored_images = autoencoder.predict(degraded_test)

# Display original, degraded, and restored images
fig, axes = plt.subplots(3, 10, figsize=(10, 3))
for i in range(10):
    axes[0, i].imshow(test_images[i])
    axes[0, i].axis('off')
    axes[1, i].imshow(degraded_test[i])
    axes[1, i].axis('off')
    axes[2, i].imshow(restored_images[i])
    axes[2, i].axis('off')
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow_datasets as tfds
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

In [ ]:
# Load pretrained VGG16 model
base_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
x = Flatten()(base_vgg16.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(100, activation='softmax')(x)
vgg16_model = Model(inputs=base_vgg16.input, outputs=x)
vgg16_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
vgg16_model.fit(train_images, train_labels, epochs=1, batch_size=64, validation_data=(test_images, test_labels))
print("VGG16 Model Evaluated:", vgg16_model.evaluate(test_images, test_labels))


 21/782 ━━━━━━━━━━━━━━━━━━━━ 47:55 4s/step - accuracy: 0.0082 - loss: 4.7903

KeyboardInterrupt: 